<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
# Unzip the data
import zipfile

with zipfile.ZipFile(r"C:\Users\Krom2\Unit4\Sprint1\DS-Unit-4-Sprint-1-NLP\module4-topic-modeling\data\Amazon-products-review.zip", 'r') as zip_ref:
    zip_ref.extractall(r"C:\Users\Krom2\Unit4\Sprint1\DS-Unit-4-Sprint-1-NLP\module4-topic-modeling\data")

In [3]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Krom2\Unit4\Sprint1\DS-Unit-4-Sprint-1-NLP\module4-topic-modeling\data\Amazon-products-review.csv")

print(df.shape)
df.head()

(28332, 24)


,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [4]:
"""
Imports
"""

import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [5]:
# Defining Stop Words
STOPWORDS = set(STOPWORDS).union(set(['Amazon', 'amazon', 'batteries']))

# Tokenize Function
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [7]:
# Apply Tokenize Function
df['tokens']=df['reviews.text'].apply(tokenize)

In [23]:
df['tokens'].head()

0    [order, item, bad, quality, missing, backup, s...
1               [bulk, expensive, way, products, like]
2                             [duracell, price, happy]
3                         [work, brand, better, price]
4                        [long, lasting, price, great]
Name: tokens, dtype: object

In [25]:
# Create term dictionary for corpus
dictionary = corpora.Dictionary(df['tokens'])

In [26]:
# Convert list of documents(corpus) into Document Term Matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['tokens']]

In [27]:
print(doc_term_matrix[0][0:10])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [28]:
# Instantiate Model
lda = gensim.models.ldamodel.LdaModel

# Train model on DTM
ldamodel = lda(doc_term_matrix, num_topics=10, id2word=dictionary, passes=50)

In [29]:
import re
words = [re.findall(r'"([^"]*)"', t[1]) for t in ldamodel.print_topics()]
topics = [' '.join(t[0:5]) for t in words]
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
love use easy gift perfect

------ Topic 1 ------
works work happy better purchase

------ Topic 2 ------
battery long life internet time

------ Topic 3 ------
great price product buy best

------ Topic 4 ------
tablet great loves old use

------ Topic 5 ------
kid camera like slow come

------ Topic 6 ------
kindle read reading light reader

------ Topic 7 ------
good price features quality far

------ Topic 8 ------
screen daughter kindle purchased bought

------ Topic 9 ------
apps store google friendly device



In [33]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [35]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.205363 -0.196237       1        1  21.627192
8      0.049026  0.150100       2        1  11.978165
3     -0.220581 -0.228843       3        1  10.043545
6      0.169118  0.062858       4        1   9.400498
5     -0.015499  0.119818       5        1   9.020758
2     -0.043379  0.144062       6        1   8.831200
7     -0.260539 -0.122944       7        1   8.214437
9      0.031229  0.161706       8        1   7.213615
0      0.266333 -0.164357       9        1   6.918581
1     -0.181072  0.073836      10        1   6.752010, topic_info=        Term         Freq        Total Category  logprob  loglift
21     great  8355.000000  8355.000000  Default  30.0000  30.0000
33      good  5162.000000  5162.000000  Default  29.0000  29.0000
4194  tablet  6796.000000  6796.000000  Default  28.0000  28.0000
18     price  4544.000000  4544.000000  Default  27.0000  27.0000
3154  kindle  3388.000000  3388.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
37       lot   259.860474   870.825012  Topic10  -4.3795   1.4860
13      like   375.885895  2591.967773  Topic10  -4.0104   0.7644
175       ve   217.673508   769.127197  Topic10  -4.5566   1.4331
18     price   302.818573  4544.537109  Topic10  -4.2265  -0.0132
471    store   211.623840  1062.195435  Topic10  -4.5848   1.0820

[522 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
56        5  0.112418       aa
56        6  0.883729       aa
57        5  0.224571      aaa
57        6  0.771168      aaa
1033      3  0.996957  ability
...     ...       ...      ...
182       2  0.547192    years
182       6  0.146383    years
1163      9  0.996068    young
3509      4  0.996105  youtube
2487      4  0.996969       yr

[686 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 4, 7, 6, 3, 8, 10, 1, 2])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling